In [150]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
import copy

setup geometry

In [151]:
geo = SplineGeometry()
geo.AddRectangle( (-3,-2), (3, 2), bcs = ("top", "out", "bot", "in"))
geo.AddCircle ( (0, 0), r=0.5, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.2))
mesh.Curve(3);
mesh_orig = copy.deepcopy(mesh)
#Draw(mesh)

setup FEM space

In [152]:
# viscosity
nu = 0.01
# Order of spaces
k = 2
# H1 vs VectorH1 -> vector field?!
V = H1(mesh,order=k, dirichlet="top|bot|cyl|in|out")
Q = H1(mesh,order=k-1)
FES = FESpace([V,V,Q]) # X = [V,Q] (without VectorH1)

setup bilinear form
velocityfield u and pressurefield p

In [153]:
ux,uy,p = FES.TrialFunction()
vx,vy,q = FES.TestFunction()

# stokes equation
def Equation(ux,uy,p,vx,vy,q):
    div_u = grad(ux)[0]+grad(uy)[1] # custom div
    div_v = grad(vx)[0]+grad(vy)[1]
    return (grad(ux)*grad(vx)+grad(uy)*grad(vy) + div_u*q + div_v*p) * dx


# (InnerProduct(grad(u),grad(v))+div(u)*q+div(v)*p)*dx
a = BilinearForm(FES)
a += Equation(ux,uy,p,vx,vy,q)
a.Assemble()

setup boundary conditions

In [154]:
gfu = GridFunction(FES)
uinf = 0.001
uin = CoefficientFunction((uinf))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("in|top|bot|out"))

x_velocity = CoefficientFunction(gfu.components[0])
scene_state = Draw(x_velocity, mesh, "vel")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

solve stokes

In [155]:
def solveStokes():
    res = gfu.vec.CreateVector()
    res.data = -a.mat * gfu.vec
    inv = a.mat.Inverse(FES.FreeDofs())
    gfu.vec.data += inv * res
    scene_state.Redraw()
solveStokes()

# Calculations/Setup for all the side constraints

Drag/ "dissipated energy" [should be working]

$J(\Omega ) = \frac{1}{2} \int_\Omega Du : Du dx$ 

In [156]:
def calc_drag(gfu):
    ux = gfu.components[0]
    uy = gfu.components[1]
    return 0.5*(grad(ux)*grad(ux)+grad(uy)*grad(uy))*dx

### get surface area of mesh (main issue)
$vol(\Omega) = \int_{\Omega} 1 \,dx \in \mathbb{R}$ should stay constant <br>
entire area: $A = 6*4=24$ <br>
$\alpha\Big(\mathrm{Vol}(\Omega(s))-\mathrm{Vol}(\Omega_0)\Big)^2$

In [157]:
alpha = 1e-4
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

def calc_surf_change():
    return alpha*(surf_t*dx-surf_0)**2

In [158]:
assert(Integrate(surf_t*dx,mesh)==Integrate(surf_t,mesh))

In [159]:
def s():
    return surf_t*dx

### barycenter (not necessarily useful yet)
$bc^\Omega = \frac{1}{vol(\Omega)}\int_{\Omega}x\,dx \in \mathbb{R}^d$ for each dimension!!

In [160]:
bc_tx = CoefficientFunction(x)
bc_0 = 1/surf_0*Integrate(bc_tx,mesh)
print(bc_0)

-1.4945112310824668e-17


In [161]:
bc_tx = CoefficientFunction(x)
bc_0 = 1/surf_0*Integrate(bc_tx,mesh)

def calc_bc_change():
    return ((1/(surf_t*dx))*(bc_t*dx)- bc_0)

In [162]:
def Cost(gfu):
    return calc_drag(gfu)

# creation of Shapediff Space + pertubation Function
function that will perturb our mesh

In [163]:
# Test and trial functions for shape derivate -> do we even need this?
VEC = H1(mesh, order=2, dim=2, dirichlet="top|bot|in|out")
PHI, X = VEC.TnT()
# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))
mesh.SetDeformation(gfset)
SetVisualization (deformation=True)

# deformation calculation
gfX = GridFunction(VEC)

In [164]:
# try for already transformed cost function:
n = specialcf.normal(2)
def TransfCost():
    #return div(X)**2*dx
    return (gfX*n)**2*ds(definedon="cyl")

# Shape Optimization
$\min_{h,u,s} \int_{\Omega(s)} \sum_{i,j=1}^2 \left( \frac{\partial u_i}{\partial x_j}\right)^2~\mathrm{d} x
 + \alpha\Big(\mathrm{Vol}(\Omega(s))-\mathrm{Vol}(\Omega_0)\Big)^2
+ \beta\sum_{j=1}^2 \Big(\mathrm{Bc}_j(\Omega(s))-\mathrm{Bc}_j(\Omega_0)\Big)^2,$

$\frac{1}{2} \int_\Omega Du : Du dx + \alpha\Big(\mathrm{Vol}(\Omega(s))-\mathrm{Vol}(\Omega_0)\Big)^2
+ \beta\sum_{j=1}^2 \Big(\mathrm{Bc}_j(\Omega(s)) - \mathrm{Bc}_j(\Omega_0)\Big)^2,$ (1)

In [165]:
# trying out different stuff for float/SumOfIntegrals Subtraction
surf_init = Parameter(surf_0)
surf_in = CoefficientFunction(surf_0)
surfdif = (CoefficientFunction(1)*dx(definedon="mesh_orig") - CoefficientFunction(1) * dx(definedon="mesh"))

In [166]:
ux = gfu.components[0]
uy = gfu.components[1]
p = gfu.components[2]

Lagrangian = Equation(ux,uy,p,ux,uy,p) + calc_drag(gfu) # stokes + drag

## Frage an Kevin Sturm:
* weiter unten genau die beschriebene Implementation aus der heutigen E-Mail
* folgendes Problem:
    * der erste Teil ist einfach ein float, aber sollte nach unserem Verständnis abhängig von dem derzeit deformierten mesh sein (siehe Zelle hier drunter)
    * über die Iterationen konvergiert es noch immer gleich/ähnlich, bis kein mesh mehr übrig ist
* wenn man das `Integrate(surf_t,mesh)` veränderlich machen will, steht man wieder vor dem vorherigen Problem
* gibt es noch weitere Möglichkeiten das zu implementieren

In [167]:
2*(Integrate(surf_t, mesh)-surf_0)

0.0

In [168]:
dJOmega = LinearForm(VEC)
dJOmega += Lagrangian.DiffShape(X)
dJOmega += 2*(Integrate(surf_t*dx, mesh)-surf_0)*s().DiffShape(X) # side constraint volume

b = BilinearForm(VEC)
b += InnerProduct(grad(X),grad(PHI))*dx + InnerProduct(X,PHI)*dx # H1-norm

In [169]:
def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

## Iteration

In [171]:
scene = Draw(gfset)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [172]:
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
a.Assemble()
solveStokes()

LineSearch = False

iter_max = 600
Jold = Integrate(calc_drag(gfu), mesh)
converged = False

# try parts of loop
mesh.SetDeformation(gfset)
scene.Redraw()

print('cost at iteration', k, ': ', Jold)
    
# input("Press enter to start optimization")
for k in range(iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    scene_state.Redraw()
    
    print('cost at iteration', k, ': ', Jold)
    #print('volume iteration', k, ': ', 24-Integrate(1,mesh))
    a.Assemble()
    solveStokes()
    
    b.Assemble()
    dJOmega.Assemble()
    SolveDeformationEquation()
    
    mesh.UnsetDeformation()
    
    #scale = 0.01 / Norm(gfX.vec)
    scale = 0.1 / Norm(gfX.vec)
    gfsetOld = gfset
    gfset.vec.data -= scale * gfX.vec
    Jnew = Integrate(calc_drag(gfu), mesh)
    
    if LineSearch:
        while Jnew > Jold and scale > 1e-12:
            #input('a')
            scale = scale / 2
            print("scale = ", scale)
            if scale <= 1e-12:
                converged = True
                break

            gfset.vec.data = gfsetOld.vec - scale * gfX.vec
            mesh.SetDeformation(gfset)
            
            a.Assemble()
            solveStokes()
            Jnew = Integrate(calc_drag(gfu), mesh)
    
    if converged==True:
        print("No more descent can be found")
        break
    Jold = Jnew

    Redraw(blocking=True)

cost at iteration 2 :  1.0997830056224484e-05
cost at iteration 0 :  1.0997830056224484e-05
cost at iteration 1 :  1.0997830056224499e-05
cost at iteration 2 :  1.0672616583769384e-05
cost at iteration 3 :  1.0374527987173267e-05
cost at iteration 4 :  1.0102205670910123e-05
cost at iteration 5 :  9.85428912742671e-06
cost at iteration 6 :  9.62943497404589e-06
cost at iteration 7 :  9.426333303636968e-06
cost at iteration 8 :  9.243721494339878e-06
cost at iteration 9 :  9.080396457157115e-06
cost at iteration 10 :  8.935226445152368e-06
cost at iteration 11 :  8.807163390029051e-06
cost at iteration 12 :  8.695256410295212e-06
cost at iteration 13 :  8.598666569000394e-06
cost at iteration 14 :  8.51668204034555e-06
cost at iteration 15 :  8.448731598775026e-06
cost at iteration 16 :  8.394392975052731e-06


KeyboardInterrupt: 